# 프로그램 붙이기

## 수치 미분

앞 절에서 정의한 `within`과 `relative`는 근사값 수열을 생성하는 다른 수치 알고리즘에서도 재사용할 수 있다.



In [16]:
within eps (a:b:rest) 
 | abs (a-b) <= eps     = b
 | otherwise            = within eps (b:rest)

relative eps (a:b:rest)
 | abs (a/b - 1) <= eps   = b
 | otherwise              = relative eps (b:rest)

그러한 예가 [수치 미분](https://en.wikipedia.org/wiki/Numerical_differentiation) 알고리즘이다.

<img src="https://upload.wikimedia.org/wikipedia/commons/1/18/Derivative.svg" width=200 />

In [17]:
easydiff f x h = (f(x + h) - f x) / h

`h`가 충분히 작으면 더 정확한 근사값을 얻을 수 있지만, 너무 작으면 라운딩 에러가 발생하여 값을 신뢰할 수 없게 된다. 적당한 `h`를 찾는 방법은 구간 `h`를 점점 줄여나가면서 `within eps` 등으로 원하는 오차 범위 내 값을 구하는 방법이다. 첫 `h0`를 적당히 큰 값으로 잡고 계산하고, 그 다음에는 구간을 반으로 줄여서 계산하고, 또 반으로 줄여서 계산하고... 

In [18]:
differentiate h0 f x = map (easydiff f x) (iterate (/2) h0)

이제 주어진 점 `x`에서의 미분 값을 계산할 수 있다.
```haskell
within eps (differentiate h0 f x)
```

In [19]:
within 1e-6 (differentiate 1 sin (pi/4))

0.7071061068563722

이 방법의 문제는 근사값 수열이 느리게 수렴한다는 점이다.

수학의 도움을 받아 개선할 수 있다. 앞서 구한 근사값 수열은 미분값에 오차항이 더해진 꼴로 볼 수 있는데, 이 때 오차항은 대략적으로 h의 지수승에 비례한다.

$정답 + h 오차항$

이때 정답을 $A$라고 하면 오차항은 $B \times 2^n \times h^n$ 라고 둘 수 있다. ($a_{i+1}$을 구할 때 $h/2$가 되므로)

$
\begin{align}
a_i &= A + B \times 2^n \times h^n \\
a_{i+1} &= A + B \times h^h
\end{align}
$

오차항을 없애면 ...

$A = \cfrac{a_{n+1} \times 2^n - a_n}{2^n - 1}$

이어진 두 근사값($a_i$와 $a_{i+1}$)이 주어지면 오차항을 제거하여 더 정확한 값($A$)을 계산할 수 있다.

In [20]:
elimerror n (a:b:rest) = (b*2^n - a) / (2^n-1) : elimerror n (b:rest)

원래의 근사값 수열에 `elimerror n`을 적용하면 더 빨리 수렴하는 수열을 얻을 수 있다. 남은 문제는 `n`을 어떻게 정하느냐 하는것. 이 값은 다음의 함수로 구할 수 있다고 한다.

In [21]:
order (a:b:c:rest) = round(logBase 2 ((a-c)/(b-c)-1))

이제 주어진 근사값 수열을 개선하는 함수를 정의할 수 있다.

In [22]:
improve s = elimerror (order s) s

In [23]:
within 1e-6 (improve (differentiate 1 sin (pi/4)))

0.7071070058051419

개선 전 후의 근사값 수열에서 첫 10개씩을 추출해보면 그 차이를 알 수 있다.

In [28]:
take 10 (differentiate 0.5 sin (pi/4))

[0.5048856975964859,0.611835119448811,0.6611301360648314,0.6845566203276636,0.6959440534591259,0.7015538499518499,0.7043374663312676,0.705723916746507,0.706415797873774,0.7067614018394579]

In [29]:
take 10 (improve (differentiate 0.5 sin (pi/4)))

[0.718784541301136,0.7104251526808518,0.7079831045904958,0.7073314865905882,0.7071636464445739,0.7071210827106853,0.7071103671617465,0.7071076790010409,0.7071070058051419,0.7071068373618346]

재미난 점은 이렇게 개선한 수열 역시 같은 속성(`h`를 계속 1/2로 줄이면서 계산한 근사값)을 가지기 때문에 또 개선할 수 있다.

In [30]:
take 10 (improve (improve (improve (differentiate 0.5 sin (pi/4)))))

[0.707102002667942,0.7071064508806535,0.707106759606181,0.7071067798094111,0.7071067810996304,0.7071067811810969,0.7071067811863712,0.7071067811865736,0.7071067811871704,0.7071067811880204]

심지어 수열 자체를 무한히 개선하는 것도 가능하다.

In [31]:
super s = map second (iterate improve s)
second (a:b:rest) = b

In [32]:
within 1e-6 (super (differentiate 0.5 sin (pi/4)))

0.7071067801878829

In [36]:
take 5 (super (differentiate 0.5 sin (pi/4)))

[0.611835119448811,0.7104251526808518,0.7071690885603772,0.7071064508806535,0.7071067801878829]

`super`는 매우 복잡한 알고리즘이지만 지연 평가 덕분에 매우 쉽게 표현할 수 있었다.